In [ ]:
#introduction and dataset description

In [ ]:
%pylab inline
#%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import calendar
import datetime
from scipy.stats import ttest_ind


In [ ]:
import pandas as pd
df = pd.read_csv('Thinkful_Chicago_Speed_Camera_Violations.csv.xz')
df.head()  #default is 5 rows returned



In [ ]:
df_clean = df.drop(columns = ['X COORDINATE', 'Y COORDINATE', 'LATITUDE', 'LONGITUDE', 'LOCATION', 'Historical Wards 2003-2015', 'Census Tracts'])  
#I chose to drop the coordinate columns as I wasn't going to use them to overlay on a map of Chicago.  Historical Wards had
#little bearing on my report.
df_clean

In [ ]:
date_split = df['VIOLATION DATE'].str.split('/', expand = True) #I want to separate out the violation date into month, date, year
violation_month = date_split[0].astype(int)
violation_day = date_split[1].astype(int)
violation_year = date_split[2].astype(int)
#Now I want to add that violation month column into the dataframe
df_clean['violation_month'] = violation_month

In [ ]:
df_clean['violation_year'] = violation_year
df_clean

In [ ]:
def return_season(x):
    x = int(x)
    if x == 6 or x == 7 or x == 8:
        return "Summer"
    elif x == 9 or x== 10 or x ==11:
        return "Fall"
    elif x == 12 or x == 1 or x == 2:
        return "Winter"
    elif x == 3 or x == 4 or x == 5:
        return "Spring"
    
df_clean['violation_season'] = df_clean['violation_month'].apply(return_season)


In [ ]:
df_clean

In [ ]:
#As of July 2019, there are 163 speed cameras in Chicago

In [ ]:
#This dataset reflects the daily volume of violations that have occurred in Children's Safety Zones for each camera. 
#The data reflects violations that occurred from July 1, 2014 until present, minus the most recent 14 days. 
#This data may change due to occasional time lags between the capturing of a potential violation and the processing and determination of a violation. 
#The most recent 14 days are not shown due to revised data being submitted to the City of Chicago. 
#The reported violations are those that have been collected by the camera and radar system and reviewed by two separate City contractors. 
#In some instances, due to the inability the registered owner of the offending vehicle, the violation may not be issued as a citation. 
#However, this dataset contains all violations regardless of whether a citation was issued, which provides an accurate view into the Automated Speed Enforcement Program violations taking place in Children's Safety Zones. 
#More information on the Safety Zone Program can be found here: http://www.cityofchicago.org/city/en/depts/cdot/supp_info/children_s_safetyzoneporgramautomaticspeedenforcement.html. The corresponding dataset for red light camera violations is https://data.cityofchicago.org/id/spqx-js37.

In [ ]:
#What is a children's safety zone?

#Safety zones are designated as a 1/8th of a mile boundary around any Chicago parks or schools. 
#The program uses enhanced signage and automated safety cameras to identify and ticket motorists who are breaking the law by exceeding the speed limits.

In [ ]:
#What is a Community Area in Chicago?
#Chicago is made up of 77 community areas that are often grouped into 9 districts or “sides.” 
#The community areas are well defined and do not overlap. 
#Each community area has one or more neighborhoods in it. The neighborhoods are often not well defined and over lap each other.

In [ ]:
#Questions to Answer:

#1.Have the overall violations dropped year over year?  #Good, per MagdalenaThat may be too simple of a question since I can answer that both in Pandas and Excel
##Future research, if # violations decreased, then is it because people drive slower? Over time can you track zip code or wards?

#2. How does violation # depend on the season? Adding the season column through Pandas to the violation data, create the function, transform
###date to season.  Logic "if date dec to march, then set to winter"


#3. How do the violation volume differ based on the areas or zip or wards of Chicago?
##3b. Which zip codes have the most violations?  Least violations?  Show top 3,5,10..

#plot histogram of violations and get mean, stdev, for the statistics.  
#Final section - proposal for further research - what did i find interesting that could be expanded on for future??
##property prices to areas of high violations


In [ ]:
#Outline
#Abstract, dataset description
##Questions
###Conclusion
####Proposal

In [ ]:
#df_clean.plot(x='VIOLATION DATE', y='VIOLATIONS', figsize=[25,12])

In [ ]:
df_clean.plot.bar(x = 'violation_year', y = 'VIOLATIONS') #This graph won't plot!!!  

In [ ]:
df_clean.hist()

In [ ]:
#numpy.random.gamma(shape, scale, size)
# Generate a gamma distribution with shape =5 and scale = 1
gamma = np.random.gamma(5,1, 100)   #shape = 5, tells me the midpoint

# Plot a histogram.
plt.hist(gamma)

# Print the histogram.
plt.show()